In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

### Задание 1

Будем парсить страницу со свежеми новостям на habr.com/ru/all/.

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

KEYWORDS = ['python', 'парсинг']

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).

В итоге должен формироваться датафрейм вида: <дата> - <заголовок> - <ссылка>

#### Дополнительная часть (необязательная)
Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.

Итоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст_статьи>

In [11]:
keywords = ['diy', 'ceo', 'api']

request = requests.get('https://habr.com/ru/all/')
parser = BeautifulSoup(request.text, 'html.parser')

In [67]:
class Post:
  def __init__(self, post, use_full_content=False):
    self.post = post
    self.use_full_content = use_full_content
    
    self.date = self.get_date()
    self.title = self.get_title()
    self.hubs = self.get_hubs()
    self.short_content = self.get_short_content()
    
    if use_full_content:
      self.full_content = self.get_full_content()
    
  def get_date(self):
    return self.post.find('span', class_='post__time').text
    
  def get_title(self):
    link = self.post.find('a', class_='post__title_link')
    return (link.get('href'), link.text)
  
  def get_hubs(self):
    hubs = self.post.find_all('li', class_='inline-list__item_hub')
    return list(map(lambda hub: hub.find('a', class_='hub-link').text, hubs))
  
  def get_short_content(self):
    content = self.post.find('div', class_='post__text')
    return content.text
  
  def get_full_content(self):
    link = self.title[0]
    request = requests.get(link)
    post_parser = BeautifulSoup(request.text, 'html.parser')
    
    content = post_parser.find('div', id='post-content-body')
    return content.text;
  
  def is_keyword_in_title(self, keyword):
    link, text = self.title
    return keyword.lower() in text.lower()
  
  def is_keyword_in_short_content(self, keyword):
    return keyword.lower() in self.short_content.lower()
  
  def is_keyword_in_full_content(self, keyword):
    return keyword.lower() in self.full_content.lower()
  
  def is_keyword_in_hubs(self, keyword):
    for hub in self.hubs:
      if keyword.lower() in hub.lower():
        return True
      
    return False
  
  def contains_keyword(self, keyword):
    is_keyword_in_title = self.is_keyword_in_title(keyword)
    is_keyword_in_hubs = self.is_keyword_in_hubs(keyword)
    
    is_keyword_in_content = self.is_keyword_in_short_content(keyword)
    if self.use_full_content:
      is_keyword_in_content = self.is_keyword_in_full_content(keyword)
    
    return (
      is_keyword_in_title or
      is_keyword_in_hubs or
      is_keyword_in_content
    )
  
  def contains_keywords(self, keywords):
    for keyword in keywords:
      if self.contains_keyword(keyword):
        return True
      
    return False
  
  def to_dataframe(self):
    data = {
      'date': self.date,
      'link': self.title[0],
      'title': self.title[1],
    }
    
    if self.use_full_content:
      data['content'] = self.full_content
    
    return pd.DataFrame([data])

In [68]:
posts = []
page_posts = parser.find_all('article', class_='post post_preview')

for bs_post in page_posts:
  post = Post(bs_post, use_full_content=True)
  if post.contains_keywords(keywords):
    posts.append(post)
    
pd_posts = pd.DataFrame()
for post in posts:
  pd_posts = pd.concat([pd_posts, post.to_dataframe()])

pd_posts

,date,link,title,content
0,вчера в 23:50,https://habr.com/ru/post/542214/,"DIY наушники. Как, а главное, зачем",Хочу поделиться своим опытом прослушивания нау...
0,вчера в 18:21,https://habr.com/ru/post/532154/,Вторая главная обязанность CEO стартапа,\n\nАли Рогани — работал в Pixar 9 лет (из них...
0,вчера в 18:13,https://habr.com/ru/post/541874/,ESP32-C3: первое знакомство. Заменим ESP8266?,Новая игрушкаВ ноябре 2020 года Espressif анон...
0,вчера в 16:55,https://habr.com/ru/company/netologyru/blog/54...,Как начинающему нарративному или левел-дизайне...,Ранее мы расспросили опытных геймдизайнеров об...
0,вчера в 16:44,https://habr.com/ru/company/otus/blog/542114/,Работа с адаптивным программируемым интерфейсо...,Перевод статьи подготовлен в преддверии старта...
0,вчера в 16:16,https://habr.com/ru/company/skillfactory/blog/...,Mockito. Из чего он приготовлен и как его пода...,"\r\nДумаю, многим довелось выпить какой-нибудь..."


### Задание 2

Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck. Список email-ов задаем переменной в начале кода:
EMAIL = [xxx@x.ru, yyy@y.com]

В итоге должен формироваться датафрейм со столбцами: <почта> - <дата утечки> - <источник утечки> - <описание утечки>

Подсказка: сервис работает при помощи "скрытого" API. Внимательно изучите post-запросы.

In [86]:
emails = ['fadil.mamedov@mail.ru', 'fadil.mamedov@gmail.com']

url = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'
data = {
  'emailAddresses': emails,
}
headers = {
  'Vaar-Header-App-Product': 'hackcheck-web-avast',
  'Vaar-Version': '0',
}

response = requests.post(url, json=data, headers=headers)
leaks = response.json()

In [117]:
pd_breaches = pd.DataFrame()

for email, value in leaks['summary'].items():
  breaches_ids = value['breaches']
  
  for breach_id in breaches_ids:
    breach = leaks['breaches'][str(breach_id)]
    
    data = {
      'email': email,
      'date': breach['publishDate'],
      'source': breach['site'],
      'description': breach['description']
    }
    
    pd_breaches = pd.concat([pd_breaches, pd.DataFrame([data])])
    
pd_breaches

,email,date,source,description
0,fadil.mamedov@mail.ru,2020-12-10T00:00:00Z,cubarecent.fun,"In November 2020, a collection of over 23,000 ..."
0,fadil.mamedov@mail.ru,2017-08-30T00:00:00Z,sprashivai.ru,"In January 2015, Sprashivai.ru's user database..."
0,fadil.mamedov@mail.ru,2017-03-15T00:00:00Z,geekedin.net,"In August 2016, tech recruiting site GeekedIn...."
0,fadil.mamedov@mail.ru,2017-06-14T00:00:00Z,nnm-club.me,"In September 2013, Russian torrent tracker NNM..."
0,fadil.mamedov@mail.ru,2017-04-10T00:00:00Z,sprashivai.ru,"In May 2015, Ask RU's user database was leaked..."
0,fadil.mamedov@mail.ru,2018-03-20T00:00:00Z,myfitnesspal.com,"In February 2018, MyFitnessPal's database was ..."
0,fadil.mamedov@mail.ru,2020-07-16T00:00:00Z,synergyregions.ru,"At an unconfirmed date, the Russian university..."
0,fadil.mamedov@mail.ru,2017-01-16T00:00:00Z,fl.ru,"In February 2015, FL.ru's entire site database..."


#### Дополнительная часть (необязательная)
Написать скрипт, который будет получать 50 последних постов указанной группы во Вконтакте.
Документация к API VK: https://vk.com/dev/methods , вам поможет метод wall.get

GROUP = 'netology'  
TOKEN = УДАЛЯЙТЕ В ВЕРСИИ ДЛЯ ПРОВЕРКИ, НА GITHUB НЕ ВЫКЛАДЫВАТЬ
В итоге должен формироваться датафрейм со столбцами: <дата поста> - <текст поста>

In [123]:
url = 'https://api.vk.com/method/wall.get'

params = {
  'domain': 'netology',
  'count': 50,
  'v': '5.103',
  'access_token': 'a0f0885394f2ec6d9adbe9036b3a9eb1f8a194ea6d670a41d5c29d100ef0dcfbb773a9ee8d1a720254dd3'
}

res = requests.get(url, params)

In [129]:
vk_posts = res.json()['response']['items']

In [156]:
pd_vk_posts = pd.DataFrame()

for vk_post in vk_posts:
  data = {
    'date': pd.to_datetime(vk_post['date'], unit='s'),
    'content': vk_post['text'],
  }
  
  pd_vk_posts = pd.concat([pd_vk_posts, pd.DataFrame([data])])

In [159]:
pd_vk_posts

,date,content
0,2021-02-10 14:13:00,"Говорят, от даты рождения зависит характер и д..."
0,2021-02-12 14:41:00,"В эфире #пятничнаябеседка. Здесь мы обсуждаем,..."
0,2021-02-12 07:59:00,В Data Science и программировании нужны люди с...
0,2021-02-11 14:59:00,"Казалось бы, ничего сложного: сиди да задавай ..."
0,2021-02-11 08:08:00,"Погоня и перестрелка с полицией в GTA, загадоч..."
0,2021-02-10 07:53:00,📆 15 февраля стартует бесплатный курс «Разрабо...
0,2021-02-09 14:57:00,"Раз, два, три — эмоция замри 🤪 \n \nЭмоциональ..."
0,2021-02-09 07:27:00,"Кто стоит за всеми приложениями, играми и сайт..."
0,2021-02-08 15:09:00,Сомнений больше нет: жизнь на удалёнке — есть ...
0,2021-02-08 07:38:00,"Если интернет сравнить с экзотической страной,..."
